In [15]:
import pandas as pd
import numpy as np
import os
import cv2
import matplotlib.pyplot as plt
import random
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Dense
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam

ERROR! Session/line number was not unique in database. History logging moved to new session 9


# Creating the Load_Data Function
To begin loading the data, let's create a variable that will represent where our dataset is stored. 
Make sure you put the letter r in front of your path string so that your computer knows that it’s supposed to read the string.

In [16]:
data_path = r"C:\Users\hqpet\Desktop\BildAuto_Hausaufgabe\UE4\archive"

def load_data(dataset):
#The images list will store the image arrays and the class list will store the class number for each image.
    images = []
    classes = []
    
    rows = pd.read_csv(dataset)
    rows = rows.sample(frac=1).reset_index(drop=True)
    
    for i, row in rows.iterrows():
        img_class = row["ClassId"]
        img_path = row["Path"]
        image = os.path.join(dataset, img_path)
        
        image = cv2.imread(image)
        image_rs = cv2.resize(image,(img_size,img_size),3)
    
        R,G,B = cv2.split(image_rs)
    
        img_r = cv2.equalizeHist(R)
        img_g = cv2.equalizeHist(G)
        img_b = cv2.equalizeHist(B)
    
        new_image = cv2.merge((img_r,img_g,img_b))
    
        if i%500 == 0:
            print(f"loaded:{i}")
        images.append(new_image)
        classes.append(img_class)
    
    X = np.array(images)
    y = np.array(classes)
    
    return (X,y)


## Defining Hyperparameters
epochs, tells the neural network how many times it should complete a full training process. In this case, the neural network will train itself 20 times (go over all 50,000 images and validate itself with 12,000 test images 20 times)!

The learning rate tells us how much the weights will be updated each time. The learning rate is often between 0 and 1.

The batch size tells us how much images the neural network will cycle through at once. It would be impossible for the computer to cycle through all 50,000 images at one go, it would crash. That’s why we have the batch size.

epochs，告诉神经网络它应该完成多少次完整的训练过程。在这种情况下，神经网络将对自己进行20次训练（对所有50,000张图像进行训练，并用12,000张测试图像对自己进行20次验证）！

学习率告诉我们权重每次将被更新多少。学习率通常在0到1之间。

批量大小告诉我们，神经网络将一次性循环处理多少图像。计算机不可能一次完成所有50,000张图像的循环，它会崩溃。这就是为什么我们有批量大小。

In [17]:
epochs = 20
learning_rate = 0.001
batch_size = 64

## Loading in the Data
First, we’ll define the paths to our test and train datasets, using the same method that we used to define the path to the dataset before

Now, we’re going to load both the training and test data in using our load_data function.

We’re going to store the images list in the variable trainX, and store the classes list in the trainY variable, and do the same for testX, and testY.

Note: This step may take a while, depending on the specs of your computer. Mine took 10–15 mins.

In [ ]:
train_data = r'C:\Users\hqpet\Desktop\BildAuto_Hausaufgabe\UE4\archive\Train.csv'
test_data = r'C:\Users\hqpet\Desktop\BildAuto_Hausaufgabe\UE4\archive\Test.csv'

(trainX, trainY) = load_data(train_data)
(testX, testY) = load_data(test_data)


ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "C:\Users\hqpet\anaconda3\envs\bga2_sose_22_220425\lib\site-packages\IPython\core\interactiveshell.py", line 3343, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-18-b528d586a026>", line 4, in <module>
    (trainX, trainY) = load_data(train_data)
  File "<ipython-input-16-459c52e6c0fb>", line 17, in load_data
    image_rs = cv2.resize(image,(img_size,img_size),3)
NameError: name 'img_size' is not defined

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\hqpet\anaconda3\envs\bga2_sose_22_220425\lib\site-packages\IPython\core\interactiveshell.py", line 2044, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'NameError' object has no attribute '_render_traceback_'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\hqpet\anaconda3\envs\bga2_sose_22

## Preparing the Data for Training
Now we’re going to normalize the data. This allows us to scale down the values in the data to be between 0 and 1, from before which was between 0 and 255.

Next, we’re going to one-hot encode the test and train labels. In essence, one-hot encoding is a way of representing each class with a binary value (1s and 0s) instead of a categorical value (“red” or “blue”). It does this by creating a diagonal matrix where the principal diagonal is ones, and the rest of the values are 0. The matrix has dimensions equal to the number of classes there are (if there are 20 classes, the matrix is a 20X20 matrix). In the matrix, each row represents a different class, so each class has its unique code. If you want to learn more about one-hot encoding, here's a great resource

And finally, we’re going to account for inequalities in the classes by assigning a weight to each class.

In [ ]:
print("UPDATE: Normalizing data")
trainX = train_X.astype("float32") / 255.0
testX = test_X.astype("float32") / 255.0
print("UPDATE: One-Hot Encoding data")
num_labels = len(np.unique(train_y))
trainY = to_categorical(trainY, num_labels)
testY = to_categorical(testY, num_labels)
class_totals = trainY.sum(axis=0)
class_weight = class_totals.max() / class_totals

## Building the Model
Before we jump into building the model, I want to point out that there is no “proper” way to build the model. There is no fixed amount of layers, dimensions or types of layers that your CNN has to have. You should play around with it to see which one gives you the best accuracy. I’ll give you the one that gave me the best accuracy.

This time, we’re going to create a class, called RoadSignClassifier (any name should do). Within the class, there is one function, createCNN, which takes 4 parameters. We’ll be using the Sequential API, which allows us to create the model layer-by-layer.

This is our first convolutional layer. We define the dimension of our output (8 X 8 X 3), and we use the activation function “relu”. We continue with this Conv2D — MaxPooling2D sequence for 2 more times.

The same thing as last time, except this time we include batch normalization. It just speeds up training.

Now we flatten the output from the final convolutional layer, perform a dropout and enter into our final dense layer. The output in the final dense layer is equal to the number of classes that we have.

That’s basically it for building the model. Time to move on ahead!

In [4]:
class RoadSignClassifier:
    def createCNN(width, height, depth, classes):
        model = Sequential()
        inputShape = (height, width, depth)
        model.add(Conv2D(8, (5, 5), input_shape=inputShape,      activation="relu"))
        model.add(MaxPooling2D(pool_size=(2, 2)))
        model.add(Conv2D(16, (3, 3), activation="relu"))
        model.add(BatchNormalization())
        model.add(Conv2D(16, (3, 3), activation="relu"))
        model.add(BatchNormalization())
        model.add(MaxPooling2D(pool_size=(2, 2)))
 
        model.add(Conv2D(32, (3, 3), padding="same", activation="relu"))
        model.add(BatchNormalization())
        model.add(Conv2D(32, (3, 3), padding="same", activation="relu"))
        model.add(BatchNormalization())
        
        model.add(Flatten())
        model.add(Dropout(0.5))
        model.add(Dense(512, activation="relu"))
        model.add(Dense(classes, activation="softmax"))
        return model

## Training the Model
Now its time for the fun part (actually this is the part where we have to wait 30 mins for the model to train lol). Its time to train our model to recognize road signs!

Here we’re performing data augmentation. Data augmentation creates modified versions of the images in our dataset. It allows us to add images to our dataset without us having to collect new ones. In Keras, we use the ImageDataGenerator module to perform data augmentation.

The first line defines our model. We use the class RoadSignClassifier, and define the width, height, depth and the number of classes.

In the second line, we create our optimizer, which in this case is the Adam optimizer. We’ll initialize the learning rate as what we had set it to be previously (0.001), we’ll also set the learning rate to decrease every epoch (that’s the decay parameter, it reduces overfitting).

The first line compiles the model. We create the model and define the optimizer, the loss, and the number of epochs.

In the second line, we fit out model (this is where the training takes place). Our data_aug.flow method applies the augmentations to our images that we defined before. The number of epochs is set to 20. For the validation data, we use our test data. The verbose is set to 1, which just means that Keras will show the progress of the model being trained as you go along.

In [5]:
#数据增强（Data Augmentation）
data_aug = ImageDataGenerator(rotation_range=10,
                              zoom_range=0.15,
                              width_shift_range=0.1,
                              height_shift_range=0.1,
                              shear_range=0.15,
                              horizontal_flip=False,
                              vertical_flip=False)


model = RoadSignClassifier.createCNN(width=32, height=32, depth=3, classes=43)
optimizer = Adam(lr=learning_rate, decay=learning_rate / (epochs))

model.compile(optimizer=optimizer, loss="categorical_crossentropy", metrics=["accuracy"])
fit = model.fit_generator(data_aug.flow(train_X, trainY, batch_size=batch_size), 
                          epochs=epochs,
                          validation_data=(test_X, testY),
                          class_weight=class_weight,
                          verbose=1)

NameError: name 'ImageDataGenerator' is not defined